# Análise de dados gerais

Este notebook possui uma análise dos dados dos resultados junto com os dados das falhas, afim de ser uma análise para realizar o modelo posteriormente.

In [ ]:
# importando dependências
import pandas as pd
import numpy as np 

In [ ]:
# carregando os dados de resultados e falhas em conjunto
df = pd.read_csv("Tabela_start.csv")

In [ ]:
df.head()

In [ ]:
# conversão para parquet
df.to_parquet('table_start_parquet.parquet')

Para conjuntos de dados muito grandes, o formato Parquet permite um acesso mais eficiente e pode ser processado em paralelo, reduzindo o tempo total de processamento e treinamento do modelo. Como Parquet armazena dados em um formato colunar, é mais eficiente para ler apenas as colunas necessárias para a análise, ao contrário de um CSV que precisa ser lido sequencialmente.

In [ ]:
df_parquet = pd.read_parquet("table_start_parquet.parquet")

### Features

Com a análise feita posteriormente e o tratamento de dados, foram feitas novas features a partir dos dados de resultados e falhas em conjunto

In [ ]:
def create_summary(df_parquet):
    def count_status(group, id_val, status_val):
        return len(group[(group['ID'] == id_val) & (group['STATUS'] == status_val)])
    
    def mean_date(group, id_val):
        return group.loc[group['ID'] == id_val, 'DATA_x'].mean()

    summary = df_parquet.groupby('KNR').apply(lambda group: pd.Series({
        'NAME': group['NAME'].iloc[0],
        'ID1 QT': len(group[group['ID'] == 1]),
        'ID1 S OK': count_status(group, 1, 10),
        'ID1 S NOK': count_status(group, 1, 13),
        'ID1 temp med': mean_date(group, 1),
        'ID2 QT': len(group[group['ID'] == 2]),
        'ID2 S OK': count_status(group, 2, 10),
        'ID2 S NOK': count_status(group, 2, 13),
        'ID2 temp med': mean_date(group, 2),
        'ID718 QT': len(group[group['ID'] == 718]),
        'ID718 S OK': count_status(group, 718, 10),
        'ID718 S NOK': count_status(group, 718, 13),
        'ID718 temp med': mean_date(group, 718),
    })).reset_index()
    
    return summary

df_summary = create_summary(df_parquet)

df_summary


In [ ]:
# salva o novo dataframe com as novas features
df_summary = df_summary.to_csv('tabela_modelo.csv')